In [1]:
from pathlib import Path

def create_folder(path):
    """
    Create a folder if it doesn't exist.
    """
    folder_path = Path(path)
    folder_path.mkdir(parents=True, exist_ok=True)

In [2]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.eld.gov.sg/finalresults2025.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table')
print(f"Found {len(tables)} tables on the page")

constituencies = []
electors_per_constituency = []
# store votes for each party
votes_casted_per_constituency = [0] * len(tables)
votes_shortfall_per_constituency = [0] * len(tables)
votes_by_party = {
    'PAP': [0] * len(tables),
    'WP': [0] * len(tables),
    'SDP': [0] * len(tables),
    'PPP': [0] * len(tables),
    'SUP': [0] * len(tables),
    'PSP': [0] * len(tables),
    'SPP': [0] * len(tables),
    'RDU': [0] * len(tables),
    'PAR': [0] * len(tables),
    'SDA': [0] * len(tables),
    'NSP': [0] * len(tables),
    'IND': [0] * len(tables),
}

for i, table in enumerate(tables):
    num_electors_elem = table.parent.find_previous_sibling('b')
    num_electors_str = num_electors_elem.get_text(strip=True)
    number_part = num_electors_str.split("Number of Electors: ")[1]
    num = int(number_part.replace(",", ""))
    electors_per_constituency.append(num)

    constituency_elem = num_electors_elem.find_previous_sibling('h3')
    constituency_name = constituency_elem.get_text(strip=True)
    constituencies.append(constituency_name)

    votes_casted = 0
    # get votes for each party
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 2:
            party_name = cells[1].get_text(strip=True)
            if party_name == '-':
                party_name = 'IND'
            
            votes = cells[2].get_text(strip=True)
            if '(' in votes:
                votes = votes.split('(')[0]
            
            if votes == 'Uncontested':
                votes_by_party[party_name][i] = num
            else:
                votes_by_party[party_name][i] = int(votes.replace(",", ""))

            votes_casted += votes_by_party[party_name][i]

    votes_casted_per_constituency[i] = votes_casted
    votes_shortfall_per_constituency[i] = num - votes_casted
# print(constituencies)
# print(electors_per_constituency)



Found 33 tables on the page


In [3]:
import pandas as pd

# Create a DataFrame with the two arrays
df = pd.DataFrame({
    'Constituency': constituencies,
    'Number of Electors': electors_per_constituency,
    'Votes Casted': votes_casted_per_constituency,
    'Votes Shortfall': votes_shortfall_per_constituency,
    'PAP': votes_by_party['PAP'],
    'WP': votes_by_party['WP'],
    'SDP': votes_by_party['SDP'],
    'PPP': votes_by_party['PPP'],
    'SUP': votes_by_party['SUP'],
    'PSP': votes_by_party['PSP'],
    'SPP': votes_by_party['SPP'],
    'RDU': votes_by_party['RDU'],
    'PAR': votes_by_party['PAR'],
    'SDA': votes_by_party['SDA'],
    'NSP': votes_by_party['NSP'],
    'IND': votes_by_party['IND'],
})

# Create a dictionary for the totals row
totals = {'Constituency': 'TOTAL'}

# Sum up each numeric column
for column in df.columns:
    if column != 'Constituency':  # Skip the non-numeric column
        totals[column] = df[column].sum()

# Create a DataFrame from the totals dictionary
totals_df = pd.DataFrame([totals])

# Concatenate the original DataFrame with the totals DataFrame
df_with_totals = pd.concat([df, totals_df], ignore_index=True)

# Save to CSV
create_folder('./out')
df_with_totals.to_csv('./out/constituency_electors.csv', index=False)

# Print votes casted percentage
print(f'Votes Casted: {df["Votes Casted"].sum() / df["Number of Electors"].sum() * 100:.2f}%')
# Print the total votes shortfall percentage
print(f'Votes Shortfall: {df["Votes Shortfall"].sum() / df["Number of Electors"].sum() * 100:.2f}%')

Votes Casted: 91.28%
Votes Shortfall: 8.72%


In [5]:
# Analyze whether the votes shortfall could have swing the result in favour of opposition (non-PAP) parties
from row_analysis import analyze_vectorized

# Add this to your notebook
analysis_results = analyze_vectorized(df)
flippable_constituencies = analysis_results[analysis_results['Could_Flip']]
print(flippable_constituencies[['Constituency', 'PAP_Vote_Share', 'Max_Opposition_Share', 'Max_Opposition_Party','Could_Flip','Votes Shortfall']])

      Constituency  PAP_Vote_Share  Max_Opposition_Share Max_Opposition_Party  \
0         ALJUNIED       40.317160             59.682840                   WP   
7          HOUGANG       37.833364             62.166636                   WP   
11      JALAN KAYU       51.469730             48.530270                   WP   
26  SEMBAWANG WEST       53.190639             46.809361                  SDP   
27        SENGKANG       43.689773             56.310227                   WP   
29        TAMPINES       52.016255             47.370529                   WP   

    Could_Flip  Votes Shortfall  
0         True            12188  
7         True             2279  
11        True             2208  
26        True             1687  
27        True             8920  
29        True            10810  
